In [1]:
import pandas as pd 
import numpy as np


In [2]:
df=pd.read_csv("clean_data_with_dates.csv")
print(df.shape)
print(df.columns)

(2190390, 20)
Index(['App Name', 'Category', 'Rating', 'Rating Count', 'Installs', 'Free',
       'Price', 'Size', 'Minimum Android', 'Developer Id', 'Released',
       'Last Updated', 'Content Rating', 'Ad Supported', 'In App Purchases',
       'Editors Choice', 'Scraped Time', 'Year', 'Age', 'Days Since Update'],
      dtype='object')


### Adding Categorical Price


In [3]:
df["Price"][df["Free"]==0].describe()

count    41950.000000
mean         5.370297
std         18.501604
min          0.204735
25%          0.990000
50%          1.990000
75%          4.490000
max        400.000000
Name: Price, dtype: float64

In [4]:
df["Price"][df["Price"]>100].nunique()

65

In [5]:
def categorize_price(price):
    if price == 0:
        return 'Free'
    elif price <= 1:
        return 'Low'
    elif price < 2:
        return 'Medium'
    elif price < 5 :
        return 'Medium High'
    elif price <100 :
        return 'High'
    else :
        return 'Very High'

df['Price Bracket'] = df['Price'].apply(categorize_price)




In [6]:
df['Price Bracket'][df['Price']>50]

1330            High
5785            High
7147            High
11467      Very High
12245           High
             ...    
2181633         High
2183388         High
2183977         High
2185056         High
2185171    Very High
Name: Price Bracket, Length: 452, dtype: object

### Adding Categorical Size

In [7]:
df['Size'].describe()

count    2.190390e+06
mean     1.916891e+01
std      2.397143e+01
min      3.125000e-03
25%      4.900000e+00
50%      1.000000e+01
75%      2.500000e+01
max      1.536000e+03
Name: Size, dtype: float64

In [8]:
df['Size'][df['Size']>100]

52         147.0
210        133.0
217        152.0
348        142.0
356        152.0
           ...  
2189565    158.0
2189651    117.0
2189665    118.0
2189788    201.0
2190061    169.0
Name: Size, Length: 27155, dtype: float64

In [9]:
def categorize_size(size):
    if size < 5  :
        return 'Very Low'
    elif size <= 10:
        return 'Low'
    elif size < 25:
        return 'Medium '
    elif size < 100 :
        return 'Medium High'
    elif size <500 :
        return 'High'
    else :
        return 'Very High'

df['Size Bracket'] = df['Size'].apply(categorize_size)


In [10]:
print(df['Size Bracket'])

0                  Low
1             Very Low
2             Very Low
3             Very Low
4                  Low
              ...     
2190385            Low
2190386    Medium High
2190387    Medium High
2190388            Low
2190389            Low
Name: Size Bracket, Length: 2190390, dtype: object


### Monetization Score =w1.In app purchases Factor. Log(Number of Installs+1) + w2.Normalized Price. Log(Number of Installs+1) + w3.Ad Supported.. Log(Number of Installs+1)


$$
\text{Normalized Price} = \frac{\log(\text{Price} + 1)}{\log(\text{Max Price} + 1)}
$$


In [11]:
import math
price=df['Price']
print(price.max())
price_normalized_up= price.apply(lambda x : math.log(x+1) )
price_normalized_down = math.log(price.max() +1)
price_normalized=(price_normalized_up/price_normalized_down)
print(price_normalized.min())
print(price_normalized.max())

400.0
0.0
1.0


In [12]:
log_of_installs=df['Installs'].apply( lambda x : math.log(x+1))
install_revenue_normalized=price_normalized*log_of_installs
i=install_revenue_normalized[install_revenue_normalized>0]
i.describe()

count    40730.000000
mean         1.276055
std          0.931375
min          0.059932
25%          0.602247
50%          1.041396
75%          1.712187
max         10.629636
dtype: float64

In [13]:
df.dtypes

App Name              object
Category              object
Rating               float64
Rating Count         float64
Installs               int64
Free                   int64
Price                float64
Size                 float64
Minimum Android       object
Developer Id          object
Released              object
Last Updated          object
Content Rating        object
Ad Supported            bool
In App Purchases        bool
Editors Choice          bool
Scraped Time          object
Year                   int64
Age                    int64
Days Since Update      int64
Price Bracket         object
Size Bracket          object
dtype: object

In [14]:
import gc
del price_normalized_down
del price_normalized_up
del price
del price_normalized
del categorize_price
del categorize_size
del i
# Force garbage collection
gc.collect()


0

In [15]:
def monetize(free):
    if free==1 :
        w_iap=0.6
        w_ads=0.4
        return (df["In App Purchases"]*log_of_installs*w_iap + w_ads*log_of_installs*df["Ad Supported"])
    else :
        w_iap=0.3
        w_ads=0.1
        w_price=0.6
        return (df["In App Purchases"]*log_of_installs*w_iap + w_ads*log_of_installs*df["Ad Supported"] + w_price*install_revenue_normalized)

df['Monetization Score'] = df['Free'].apply(monetize)

print(df['Monetization Score'])

MemoryError: Unable to allocate 16.7 MiB for an array with shape (2190390,) and data type float64

In [ ]:
%who

In [16]:
df.columns

Index(['App Name', 'Category', 'Rating', 'Rating Count', 'Installs', 'Free',
       'Price', 'Size', 'Minimum Android', 'Developer Id', 'Released',
       'Last Updated', 'Content Rating', 'Ad Supported', 'In App Purchases',
       'Editors Choice', 'Scraped Time', 'Year', 'Age', 'Days Since Update',
       'Price Bracket', 'Size Bracket'],
      dtype='object')

In [ ]:
%who